In [3]:
# Before Running this tool, you need to set up the "settings.json" file.
# You will also need a oauth token to connect to the account you wish to use for the bot.
# Log on to your bot account, and then use this tool to generate a token: 
#    https://twitchapps.com/tmi/
# Add your information in the required fields in the .json file.


# You can turn off the toolbar above in the "View" Menu.
# Activate each cell in order by clicking in the cell and pressing Shift+Enter.

In [ ]:
!pip install -r requirements.txt

In [ ]:
import IPython
from IPython.display import update_display, display, HTML, Javascript, clear_output
import os
import time
import re
from VoteBot import VoteBot

class DisplayVoteBot(VoteBot):
    def __init__(self, autovote=False):
        style_path = os.getcwd() + "/vote_panel_style.css"
        styling = open(style_path, "r", encoding="utf-8").read()
        bg_div = HTML("<div id=background-div></div>")
        html_path = os.getcwd() + "/output_layout.html"
        
        html = open(html_path, "r", encoding="utf-8").read() + "<style>" + styling + "</style>"
        display(HTML(html), display_id="vote_table")
        display(bg_div)
        display(Javascript(""), display_id="js")
        self.vote_log = ""
        super().__init__(autovote=autovote)

    def change_time(self, time="&nbsp"):
        # changes the time display
        display(Javascript(
            "document.getElementById('vote-time-cell').innerHTML = '" + time + "';"),
            update=True, display_id="js")
        
    def change_prompt(self, prompt="", prompt_class="stopped-prompt", show_notif=True):
        # changes the class and text of the prompt table
        prompt = re.escape(prompt)
        js = "document.getElementById('main-panel').className = '" + prompt_class + "';"
        js += "document.getElementById('title-row').className = '" + prompt_class + "-title';"
        js += "document.getElementById('title-cell').innerHTML = '" + prompt + "';"
        if show_notif:
            if prompt_class == "collecting-prompt":
                notif = "Use !v (suggestion)<br>(between " + str(self.min_msg_size) + " to " + str(self.max_msg_size) + " characters)"
            elif prompt_class == "voting-prompt":
                notif = "Type the number to vote!"
            elif prompt_class == "stopped-prompt":
                notif = "No active vote"
            elif prompt_class == "cooldown-prompt":
                notif = "Vote cooldown<br>Prepare for next round!"
            else:
                notif = ""
                
            js  += "document.getElementById('notif-cell').innerHTML = '" + notif + "';"
        else:
            js  += "document.getElementById('notif-cell').innerHTML = '';"
            
        display(Javascript(js), update=True, display_id="js")
    
    def display_vote_start(self, prompt=None, prompt_class="collecting-prompt"):
        # called on vote start to display the beginning of collection
        if prompt == None:
            prompt = self.curr_prompt
        if self.sending_message:
            self.ws.send_message("Starting vote: " + prompt)
            
        self.change_prompt(prompt, prompt_class)
        self.clear_vote_table()
        self.change_time()
        
    def clear_vote_table(self):
        js = Javascript("document.getElementById('vote-table').innerHTML='';")
        display(js, update=True, display_id="js")
        
    def display_vote_stop(self):
        # displays voting has ended while keeping the results
        self.change_prompt(self.curr_prompt)
        self.change_time()
        
    def display_final_results(self):
        # sends winning candidate message and does final update of voting table
        winner = self.get_winner()
        if winner:
            winner_msg = str(winner[0][1]) + " votes: " + winner[0][0]
            self.display_collected_rows(winner[1])
            if self.sending_message:
                self.ws.send_message(winner_msg)

    def display_collected_rows(self, winner_num=-1, skip_voting=False):
        # update list when it becomes updated
        table_layout = ""
        for num in range(len(self.commands_collected)):
            selected = self.commands_collected[num]
            index = str(num + 1) + ")"
            if bool(selected[2]):
                item = [index, selected[0], selected[1]]
            else:
                # if selection has been banned, clear the row text
                item = ["", "", ""]
            if num == winner_num:
                # if on winning row, change the cell class
                row = "<tr class='selected-row'><td><div class='index-cell'>" + item[0] + "</div>"
            else:
                row = "<tr class='candidate-row'><td><div class='index-cell'>" + item[0] + "</div>"

            row += "<div class='candidate-cell'>" + item[1] + "</div>"
            if not skip_voting:
                row += "<div class='vote-div'>" + str(item[2]) + "</div>"
                
            table_layout += row + "</td></tr>"

        table_layout = re.escape(table_layout)
        js = "document.getElementById('vote-table').innerHTML='" + table_layout + "';"
        js = Javascript(js)
        display(js, update=True, display_id="js")
        self.updated.value = True
        
    def display_clear(self):
        # stop voting, and clear all tables
        self.change_prompt()
        self.change_time()
        self.updated.value = True
        self.clear_vote_table()

    def wait_for_updates(self, duration, mode, prompt_class, use_delay=True, skip_voting=False):
        # loop that waits for each phase, and updates 
        self.change_prompt(self.curr_prompt, prompt_class)
        wait_time = duration
        if use_delay:
             wait_time += self.stream_delay
        i = 0
        while i <= wait_time and self.curr_mode.value == mode:
            if not self.updated.value:
                self.display_collected_rows(skip_voting=skip_voting)
            if i <= duration:
                self.change_time(str(duration - i))
            else:
                self.change_time("DELAY")
            time.sleep(1)
            i += 1
        if self.curr_mode.value != b'l': # do last update if not cleared
            self.display_collected_rows()

In [ ]:
bg_color = "#00f" # bg_color sets the background color to be keyed out in streams 
display(HTML("<style>#notebook-container { width: 100%; }div.output_html { background-color: " + bg_color + " }</style>"))
DisplayVoteBot(autovote=True)